<a href="https://colab.research.google.com/github/vifirsanova/llm-tutorial/blob/main/colab_groq_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 60‑минутный Colab‑туториал: Groq API + RAG + Telegram  

- Быстрый старт с Groq API  
- Углублённый RAG: FAISS + sentence‑transformers + LangChain (Groq как LLM provider)  
- Поиск в БД и базовая валидация ответа  

## Важно — перед запуском
1. Этот ноутбук **не** использует локальные модели — все генерации идут через Groq API.  
2. Вам понадобится Groq API ключ (создайте в консоли Groq Cloud).  
3. Для Telegram‑бота нужен токен бота (через @BotFather).  
4. Всё будет выполняться прямо в Colab — однако для работы бота вам нужно держать ноутбук запущенным (или деплоить на сервер).

## 1) Установка зависимостей

In [1]:
!pip install --quiet groq langchain langchain-groq sentence-transformers faiss-cpu tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 23.5 MB/s eta 0:00:00


## 2) Быстрый старт с Groq API

In [2]:
import os, getpass
from google.colab import userdata
from groq import Groq
from IPython.display import Markdown

try:
    os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
except Exception as e:
    os.environ['GROQ_API_KEY'] = getpass.getpass('Введите ваш GROQ_API_KEY (скрыто): ')

client = Groq(api_key=os.environ['GROQ_API_KEY'])

def groq_chat(prompt, model='meta-llama/llama-4-scout-17b-16e-instruct'):
    messages = [{'role': 'user', 'content': prompt}]
    resp = client.chat.completions.create(model=model, messages=messages, max_tokens=300, temperature=0.2)
    return resp.choices[0].message.content

display(Markdown(groq_chat("Объясни простыми словами, что такое HTML.")))

**HTML** (HyperText Markup Language) — это язык разметки гипертекста, который используется для создания веб-страниц и других онлайн-документов. Проще говоря, это способ рассказать компьютеру, как отображать текст, изображения и другие элементы на веб-странице.

Представь, что ты пишешь письмо на бумаге. Ты используешь разные цвета чернил, шрифты и форматирование, чтобы сделать текст более читаемым и интересным. HTML работает примерно так же, но вместо чернил и бумаги он использует специальные команды (теги), чтобы сказать компьютеру, как отображать контент на веб-странице.

Например, если ты хочешь сделать текст жирным, ты можешь использовать тег `<b>`. Если ты хочешь добавить изображение, ты можешь использовать тег `<img>`. HTML состоит из множества таких тегов, которые позволяют создавать разные элементы на веб-странице.

Когда ты создаешь веб-страницу с помощью HTML, компьютер интерпретирует эти теги и отображает контент на экране в соответствии с твоими инструкциями. Это позволяет создавать веб-страницы с разным содержимым, оформлением и интерактивными элементами.

В целом, HTML — это фундаментальный язык для создания веб-страниц и онлайн-документов, который позволяет разработчикам создавать и структурировать контент для интернета.

## 3) Управление диалогом с историей сообщений

In [6]:
# Простейший менеджер диалога (подходит для небольших сессий)
dialogue = []

def add_message(role, text):
    dialogue.append({'role': role, 'content': text})

def get_reply(user_text):
    add_message('user', user_text)
    max_messages = 10
    history = dialogue[-max_messages:]

    # Формируем промпт с историей
    prompt_parts = []
    for msg in history:
        prompt_parts.append(f"{msg['role']}: {msg['content']}")
    prompt_parts.append("assistant:")

    full_prompt = "\n".join(prompt_parts)

    # Получаем ответ от модели
    response = groq_chat(full_prompt)

    # Добавляем ответ в историю
    add_message('assistant', response)

    return response

# Пример использования
add_message('system', 'Ты полезный ассистент по веб-разработке. Отвечай кратко и по делу.')

In [14]:
# Тестируем диалог
msg = None
while msg != 'exit':
  msg = input('User: ')
  print('Assistant:')
  display(Markdown(get_reply(msg)))
  print('\n')

User: Привет!
Assistant:


Серьезно? Еще раз привет? Окей, ладно... Привет! 

Давай попробуем иначе: как я могу тебе помочь сегодня?



User: Расскажи про HTML
Assistant:


HTML (HyperText Markup Language) — это стандартный язык разметки гипертекста, используемый для создания веб-страниц. Он является основой для любого веб-сайта и используется для определения структуры и содержания веб-страниц.

**Основные элементы HTML:**

1. **Теги** (tags): HTML состоит из тегов, которые используются для разметки элементов на странице. Теги обычно парные, т.е. имеют открывающий и закрывающий теги.
2. **Элементы** (elements): Элементы HTML — это то, что содержится внутри тегов. Например, текст, изображения, ссылки и т.д.
3. **Атрибуты** (attributes): Атрибуты — это дополнительные свойства элементов, которые определяют их поведение или внешний вид.

**Некоторые распространенные теги HTML:**

1. **`<html>`**: Корневой элемент HTML-документа.
2. **`<head>`**: Содержит метаданные о документе, такие как заголовок, кодировка и ссылки на стили.
3. **`<body>`**: Содержит содержимое веб-страницы.
4. **`<h1>`-`<h6>`**: Заголовки разных уровней.
5. **`<p>`**: Абзац текста.
6. **`<img>`**: Изображение.
7. **`<a>`**: Ссылка.

**Преимущества HTML:**

1. **Универсальность**: HTML — это стандартный язык, понимаемый всеми браузерами



User: Какую роль играет Pug в HTML
Assistant:


Pug (ранее известный как Jade) — это язык шаблонов для Node.js, который позволяет писать HTML-код в более краткой и читаемой форме. Pug не является заменой HTML, а скорее надстройкой над ним, позволяющей упростить процесс создания HTML-документов.

**Роль Pug в HTML:**

1. **Упрощение HTML-кода**: Pug позволяет писать HTML-код в более краткой форме, используя indentation (отступы) для определения структуры документа.
2. **Автоматическая генерация HTML**: Pug компилирует написанный код в чистый HTML, что позволяет разработчикам не писать однообразный и повторяющийся код.
3. **Использование переменных и выражений**: Pug позволяет использовать переменные и выражения для динамической генерации контента.
4. **Включение миксов и функций**: Pug поддерживает возможность включения миксов (смежных фрагментов кода) и функций для повторного использования кода.

**Пример Pug-кода:**
```pug
html
  head
    title Страница
  body
    h1 Привет, мир!
    p Это параграф текста
```
Этот Pug-код компилируется в следующий HTML-код:
```html
<html>
  <head>
    <title>Страница</title>
  </head>
  <body>
    <h1>Привет, мир!</h1>
    <p>Это параграф текста</p>
  </body>
</html>




User: О чем мы общались?
Assistant:


Мы общались о:

1. Приветствовали друг друга 
2. Обсуждали HTML (HyperText Markup Language) и его основные элементы, теги, атрибуты и распространенные теги.
3. Обсуждали роль Pug (Jade) в HTML, как языка шаблонов для Node.js, который упрощает написание HTML-кода и позволяет автоматическую генерацию HTML.



User: exit
Assistant:


Всего хорошего!

## 4) Углублённый RAG (Retrieval‑Augmented Generation)

**Идея:** использовать базу знаний (например, документацию по веб‑разработке), искать релевантные фрагменты и подставлять их в контекст ответа.  
Мы будем использовать:
- Sentence‑Transformers для эмбеддингов,  
- FAISS для поиска ближайших соседей,  
- LangChain + Groq для генерации ответа.  


In [21]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from langchain_groq import ChatGroq
from langchain import PromptTemplate, LLMChain

# Документы (простейшие подсказки по веб‑разработке)
docs = [
    "HTML используется для структуры веб‑страницы. Теги определяют элементы: заголовки, параграфы, ссылки.",
    "C помощью CSS можно создавать полноценные анимационные элементы и даже целые композиции без использования JavaScript.",
    "JavaScript позволяет добавить интерактивность на сайт: обработка событий, анимации, AJAX-запросы.",
    "Python часто используется на сервере (например, с Django или Flask) для обработки запросов и работы с базами данных.",
    "React - популярная JavaScript библиотека для создания пользовательских интерфейсов с компонентным подходом.",
    "Базы данных хранят информацию. SQL базы (MySQL, PostgreSQL) и NoSQL (MongoDB) имеют разные подходы к хранению данных.",
    "API (Application Programming Interface) позволяет программам общаться друг с другом. REST API использует HTTP методы.",
    "Веб-безопасность включает HTTPS, валидацию входных данных, защиту от XSS и CSRF атак."
]

# Эмбеддинги
emb_model = SentenceTransformer('all-MiniLM-L6-v2')
doc_embs = emb_model.encode(docs, convert_to_numpy=True)

# FAISS
d = doc_embs.shape[1]
index = faiss.IndexFlatL2(d)
index.add(doc_embs)

def retrieve(query, k=2):
    q_emb = emb_model.encode([query], convert_to_numpy=True)
    D, I = index.search(q_emb, k=k)
    return [docs[i] for i in I[0]]

# Groq LLM
llm = ChatGroq(model='meta-llama/llama-4-scout-17b-16e-instruct')

prompt = PromptTemplate(
    input_variables=['context','question'],
    template="""Ты — помощник веб‑разработчика. Используй контекст и ответь на вопрос.
Контекст:
{context}

Вопрос: {question}
Ответ (кратко и понятно):"""
)
chain = LLMChain(llm=llm, prompt=prompt)

def rag_answer(query):
    context = "\n".join(retrieve(query, k=2))
    return chain.run({'context': context, 'question': query})

print(rag_answer("Что делает CSS?"))

CSS позволяет создавать анимационные элементы и композиции для веб-страниц.


## 5) Проверка и базовая валидация ответа
Чтобы убедиться, что ответ модели корректный:
- Проверяем, что строка непустая.  
- Можно добавить правила: ответ не слишком длинный, не содержит запрещённых слов.  
Простейший пример ниже.

In [32]:
def validate_answer(answer: str) -> str:
    forbidden_words = ['HTML', 'HyperText', 'Markup']
    if any(word in answer.split() for word in forbidden_words):
        return "Извините, я не могу ответить на этот вопрос."
    return answer

# Пример
raw = rag_answer("Расскажи про HTML.")
print("Ответ:", raw)
print("Валидированный:", validate_answer(raw))

Ответ: HTML (HyperText Markup Language) — стандартный язык разметки гипертекста, который используется для создания структуры и содержания веб-страниц. С помощью HTML определяются элементы веб-страницы, такие как заголовки, параграфы, ссылки, изображения и другие. HTML состоит из набора тегов, которые заключаются в угловые скобки и используются для wrapping контента и придания ему определенного смысла.
Валидированный: Извините, я не могу ответить на этот вопрос.


In [34]:
# Пример с LangChain (упрощённо)
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

template = """Ты — полезный ассистент.
Вопрос: {question}
Ответ:"""
prompt = PromptTemplate(input_variables=['question'], template=template)

chain = LLMChain(llm=llm, prompt=prompt)

def langchain_answer(question):
    return chain.run(question)

In [36]:
display(Markdown('LangChain: пример создания LLMChain и PromptTemplate.'))
display(Markdown(langchain_answer('Как организовать заметки по проекту?')))

LangChain: пример создания LLMChain и PromptTemplate.

Существует множество способов организовать заметки по проекту, и лучший подход будет зависеть от ваших личных предпочтений, типа проекта и количества заметок, которые вы делаете. Вот несколько популярных методов:

1. **Папка или Binder**: Храните все свои заметки в физической папке или binder, помеченной названием проекта. Это может включать бумажные заметки, встречи, электронные письма и другие соответствующие документы.

2. **Цифровые приложения для заметок**: Используйте приложения для заметок, такие как Evernote, OneNote, Simplenote или Google Keep. Эти приложения позволяют вам создавать, организовывать и искать заметки по ключевым словам, тегам или папкам.

3. **Системы тегов или категорий**: Организуйте свои заметки с помощью системы тегов или категорий. Например, вы можете использовать теги для разных тем, таких как «Встречи», «Задачи», «Идеи» или «Проблемы». 

4. **Хронологическая организация**: Храните свои заметки в хронологическом порядке, с самыми новыми заметками сверху или снизу. Это особенно полезно для отслеживания прогресса со временем.

5. **Шаблоны для встреч**: Создайте шаблон для заметок во время встреч, который может включать поля для даты, участников, повестки дня и выводов. Это помогает обеспечить, что вы всегда фиксируете ключевую информацию.

6. **Инструменты управления проектами**: Многие инструменты управления проектами, такие как Trello, Asana или Notion, позволяют интегрировать заметки прямо в задачи или использовать их как отдельный документ в контексте проекта.

7. **Заметки, организованные по действиям**: Организуйте свои заметки на основе необходимых действий или решений. Например, вы можете иметь список «К исполнению», «К обсуждению» и «К сведению».

8. **Карточки или стикеры**: Используйте физические карточки или стикеры для быстрых заметок и организуйте их на доске или стене. Это полезно для визуального и тактильного подхода к заметкам.

Независимо от выбранного метода, важно регулярно просматривать и 정리вать свои заметки, чтобы информация оставалась актуальной и легко доступной.

In [40]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

model = SentenceTransformer('all-MiniLM-L6-v2')

texts = [
    'Как приготовить борщ: основные шаги.',
    'Инструкция по установке Python на Windows.',
    'Советы по продуктивности при работе из дома.',
    'Основы веб-разработки: HTML, CSS, JavaScript.',
    'Как настроить виртуальное окружение в Python.',
    'Базы данных: SQL vs NoSQL, основные различия.',
    'Введение в машинное обучение для начинающих.',
    'Как создать REST API на Flask.'
]

# Создаем эмбеддинги
embeddings = model.encode(texts, convert_to_numpy=True)

# Создаем FAISS индекс
d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(embeddings)

# Поиск похожих текстов
query = "веб-разработка"
q_emb = model.encode([query], convert_to_numpy=True)
D, I = index.search(q_emb, k=3)
print('Результаты поиска (индексы похожих текстов):', I[0])
for idx in I[0]:
    print('-', texts[idx])

Результаты поиска (индексы похожих текстов): [3 2 6]
- Основы веб-разработки: HTML, CSS, JavaScript.
- Советы по продуктивности при работе из дома.
- Введение в машинное обучение для начинающих.


## 6) Практические советы
- **Ключи:** храните GROQ_API_KEY и TELEGRAM_BOT_TOKEN в переменных окружения.  
- **Деплой:** для постоянной работы бота используйте сервер или хостинг (например, Railway/Render).  
- **RAG:** используйте свои документы (FAQ, мануалы), чтобы бот отвечал именно по вашей базе знаний.  
- **Валидация:** всегда проверяйте, что ответы модели адекватны (пустые, слишком длинные или не по теме — фильтруйте).  
